
# Final Assignment (Edited, Minimal)
This notebook includes only the required steps:

1) Tesla stock (yfinance)  
2) Tesla revenue (webscrape)  
3) GameStop stock (yfinance)  
4) GameStop revenue (webscrape)  
5) Tesla dashboard (stock + revenue)  
6) GameStop dashboard (stock + revenue)  
7) Share-ready note

> Run from top to bottom. If packages are missing, use the optional install cell.


In [9]:

# Optional installs (uncomment if needed)
# !pip install yfinance requests beautifulsoup4 lxml plotly pandas --quiet


In [10]:

import pandas as pd
import yfinance as yf
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

pd.set_option("display.max_columns", 50)
print("Ready.")


Ready.


In [11]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

def scrape_macrotrends_quarterly_revenue_simple(url: str) -> pd.DataFrame:
    """
    Returns a DataFrame with columns:
      - Date (YYYY-MM-DD)
      - Revenue (Millions USD, numeric)
      - Revenue_Billions (optional convenience column)
    It finds the table whose <th> contains 'Quarterly Revenue'.
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                      "(KHTML, like Gecko) Chrome/124.0 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://www.google.com/"
    }
    html = requests.get(url, headers=headers, timeout=30).text
    soup = BeautifulSoup(html, "lxml")

    # Find the specific revenue table under your div: <table class="historical_data_table">
    table = None
    for tbl in soup.select("table.historical_data_table"):
        th = tbl.find("th")
        if th and "Quarterly Revenue" in th.get_text(strip=True):
            table = tbl
            break
    if table is None:
        raise ValueError("Quarterly Revenue table not found.")

    # Parse just that table into a DataFrame
    df = pd.read_html(str(table))[0]
    df.columns = ["Date", "Revenue"]  # (Date, $xx,xxx in Millions)
    df = df.dropna(subset=["Date", "Revenue"]).copy()

    # Clean "$" and commas; keep **Millions USD** as numbers
    df["Revenue"] = (
        df["Revenue"].astype(str)
        .str.replace("$", "", regex=False)
        .str.replace(",", "", regex=False)
        .str.strip()
    ).astype(float)

    # Optional: convenient billions column for plotting if you prefer
    df["Revenue_Billions"] = df["Revenue"] / 1000.0
    return df


## Q1 — Tesla (TSLA) stock via yfinance

In [12]:

tsla = yf.Ticker("TSLA")
tesla_data = tsla.history(period="max")
tesla_data = tesla_data.reset_index()
tesla_data.head()


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


## Q2 — Tesla revenue via webscraping

In [13]:

tesla_url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
tesla_revenue = scrape_macrotrends_quarterly_revenue_simple(tesla_url)
tesla_revenue.head()


/tmp/ipykernel_18028/1938386790.py:33: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


,Date,Revenue,Revenue_Billions
0,2025-06-30,22496.0,22.496
1,2025-03-31,19335.0,19.335
2,2024-12-31,25707.0,25.707
3,2024-09-30,25182.0,25.182
4,2024-06-30,25500.0,25.500


## Q3 — GameStop (GME) stock via yfinance

In [14]:

gme = yf.Ticker("GME")
gamestop_data = gme.history(period="max")
gamestop_data = gamestop_data.reset_index()
gamestop_data.head()


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620128,1.693349,1.603295,1.691666,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658002,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662209,1.603296,1.662209,6892800,0.0,0.0


## Q4 — GameStop revenue via webscraping

In [15]:

import pandas as pd
import requests
from bs4 import BeautifulSoup

def scrape_macrotrends_quarterly_revenue_simple(url: str) -> pd.DataFrame:
    """
    Returns a DataFrame with:
      - Date (YYYY-MM-DD)
      - Revenue (Millions USD, numeric)
      - Revenue_Billions (helper = Revenue/1000)
    Picks the 'Quarterly Revenue' table inside the #style-1 container.
    Minimal, no over-engineering.
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                      "(KHTML, like Gecko) Chrome/124.0 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://www.google.com/"
    }
    html = requests.get(url, headers=headers, timeout=30).text
    soup = BeautifulSoup(html, "lxml")

    # 1) Look inside the main scrollable container
    box = soup.select_one("#style-1") or soup
    tables = box.select("table.historical_data_table")

    # 2) Prefer the table whose header contains 'Quarterly Revenue';
    #    fallback to the last table (quarterly is usually on the right).
    table = None
    for tbl in tables:
        th = tbl.find("th")
        if th and "Quarterly Revenue" in th.get_text(strip=True):
            table = tbl
            break
    if table is None and tables:
        table = tables[-1]
    if table is None:
        raise ValueError("Quarterly Revenue table not found.")

    # 3) Parse and clean
    df = pd.read_html(str(table))[0]
    df.columns = ["Date", "Revenue"]
    df = df.dropna(subset=["Date", "Revenue"]).copy()
    df["Revenue"] = (df["Revenue"].astype(str)
                     .str.replace("$", "", regex=False)
                     .str.replace(",", "", regex=False)
                     .str.strip())
    df = df[df["Revenue"] != ""].copy()
    df["Revenue"] = pd.to_numeric(df["Revenue"], errors="coerce")
    df = df.dropna(subset=["Revenue"]).reset_index(drop=True)

    # helper for plotting in billions if you want
    df["Revenue_Billions"] = df["Revenue"] / 1000.0
    return df


gme_url = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"
gamestop_revenue = scrape_macrotrends_quarterly_revenue_simple(gme_url)
gamestop_revenue.head()


/tmp/ipykernel_18028/2097682705.py:41: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


,Date,Revenue,Revenue_Billions
0,2025-07-31,972,0.972
1,2025-04-30,732,0.732
2,2025-01-31,1283,1.283
3,2024-10-31,860,0.860
4,2024-07-31,798,0.798



## Q5 — Tesla stock & revenue dashboard

In [16]:
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def make_dashboard(stock_df, revenue_df, ticker, title):
    # Ensure dates exist
    s = stock_df.copy()
    if "Date" not in s.columns:
        s = s.reset_index()
    s["Date"] = pd.to_datetime(s["Date"], errors="coerce")

    r = revenue_df.copy()
    r["Date"] = pd.to_datetime(r["Date"], errors="coerce")

    # Choose Millions or Billions automatically
    y = r["Revenue_Billions"] if "Revenue_Billions" in r.columns else r["Revenue"]
    y_label = "Revenue (Billions USD)" if "Revenue_Billions" in r.columns else "Revenue (Millions USD)"

    fig = make_subplots(rows=2, cols=1, shared_xaxes=False, vertical_spacing=0.1,
                        subplot_titles=(f"{ticker} Close Price", f"{ticker} Quarterly Revenue"))
    fig.add_trace(go.Scatter(x=s["Date"], y=s["Close"], mode="lines", name="Close"), row=1, col=1)
    fig.add_trace(go.Bar(x=r["Date"], y=y, name="Revenue"), row=2, col=1)

    fig.update_yaxes(title_text="Close (USD)", row=1, col=1)
    fig.update_yaxes(title_text=y_label, row=2, col=1)
    fig.update_layout(height=700, title=title, showlegend=False)
    fig.show()

make_dashboard(tesla_data, tesla_revenue, "TSLA", "Tesla — Stock & Quarterly Revenue")


## Q6 — GameStop stock & revenue dashboard

In [17]:

make_dashboard(gamestop_data, gamestop_revenue, "GME", "GameStop — Stock & Quarterly Revenue")



## Q7 — Share
Export this notebook as HTML or upload to a public repo (e.g., GitHub) and submit the link.
